# TAREA 2 (23-24) ALGORITMOS EVOLUTIVOS
Aarón Medina Melián

<p>La distancias entre las ciudades están obtenidas de: https://es.mejoresrutas.com/matrix/custom/DD66C588-B55C-F95C-89C4-76D158D5578C/</p>
<p>Para intentar realizarlo lo más realista, se tratan de distancias de conducción entre ciudades en kilómetros.</p>

|                  | Las Palmas de GC | Telde | Arucas | Agaete | Mogán |
|:----------------:|:----------------:|:-----:|:------:|:------:|:-----:|
| Las Palmas de GC | 0                | 20    | 13     | 33     | 83    |
| Telde            | 20               | 0     | 26     | 51     | 68    |
| Arucas           | 13               | 26    | 0      | 27     | 89    |
| Agaete           | 33               | 51    | 27     | 0      | 53    |
| Mogán            | 83               | 68    | 89     | 53     | 0     |

Importación de las librerías necesarias

In [54]:
import csv
import random
import numpy as np
import requests
from io import StringIO

<p>Funciones para cargar los csv de distancias y los nombres de las ciudades</p>
<p>Utilizo la librería requests para hacer peticiones a las url raw de github para poder montar los arrays puros de Python sin uso de otras librerías como pandas</p>

In [55]:
# URL del archivo CSV
url_archivo = "https://raw.githubusercontent.com/aaronmed/m2d/master/TAREA%202%20(23-24)%20ALGORITMOS%20EVOLUTIVOS/distancias.csv"

# Descargar el contenido del archivo CSV desde la URL
response = requests.get(url_archivo)
csv_content = StringIO(response.text)

# Procesar el contenido CSV
distancias = []

lector_csv = csv.reader(csv_content)

for fila in lector_csv:
    fila_numerica = []
    for valor in fila:
        valor_numerico = int(valor)
        fila_numerica.append(valor_numerico)

    distancias.append(fila_numerica)

print(distancias)


[[0, 20, 13, 33, 83], [20, 0, 26, 51, 68], [13, 26, 0, 27, 89], [33, 51, 27, 0, 53], [83, 68, 89, 53, 0]]


In [56]:
url = 'https://raw.githubusercontent.com/aaronmed/m2d/master/TAREA%202%20(23-24)%20ALGORITMOS%20EVOLUTIVOS/ciudades.csv'
ciudades = []

# Descargar el contenido del archivo CSV desde la URL
response = requests.get(url)
csv_content = StringIO(response.text)

# Leer el archivo CSV
csv_reader = csv.reader(csv_content)

# Obtener la primera fila del CSV
row = next(csv_reader)

# Extender la lista de ciudades con los datos de la primera fila
ciudades.extend(row)

print(ciudades)

['Las Palmas de Gran Canaria', 'Telde', 'Arucas', 'Agaete', 'Mogán']


### Función fitness
<div>Calcula la distancia total de un recorrido representado por "poblacion", utilizando la matriz "distancias"</div>
<div>que hemos cargado previamente mediante un .csv</div>

In [57]:
def fitness(poblacion):
    distancia_total = 0
    for i in range(len(poblacion)-1):
        distancia_total += distancias[poblacion[i]][poblacion[i+1]]
    distancia_total += distancias[poblacion[len(poblacion)-1]][poblacion[0]]
    return distancia_total

### Función de mutación
<div>Elegimos dos posiciones aleatorias para sustituirlas la población</div>
<div>Nos aseguramos que las dos posiciones sean distintas para que se pueda realizar la mutación correctamente</div>
<div>Sutituimos una posición por otra</div>

In [58]:
def mutacion(poblacion):
    random_position1 = random.randint(0, len(poblacion)-1)
    random_position2 = random.randint(0, len(poblacion)-1)
    while random_position1 == random_position2:
        random_position2 = random.randint(0, len(poblacion)-1)
    poblacion[random_position1], poblacion[random_position2]  = poblacion[random_position2], poblacion[random_position1]
    return poblacion

### Función de recombinación
<div>Primero creamos un nuevo_individuo y vamos recorriendo las poblaciones</div>
<div>Utilizamos los índices de la primera población (poblacion1) para seleccionar elementos de la segunda población (poblacion2).</div>

In [59]:

def recombinacion(poblacion1, poblacion2):
    new_poblacion = np.zeros(len(poblacion1), dtype=int)

    for i in range(len(poblacion1)):
        new_poblacion[i] = poblacion2[poblacion1[i]] 
    return new_poblacion

### Método que busca las mejores poblaciones
<div>Busca las M(numero_mejores) mejores poblaciones dada una poblacion</div>
<ul>
    <li> si el tamaño de poblaciones es igual al número de mejores, devolvemos poblaciones, ya que no se puede buscar mejor numero de poblaciones
</ul>

In [60]:
def buscar_mejores_poblaciones(poblaciones, numero_mejores):
    if len(poblaciones) == numero_mejores:
        return poblaciones
    
    if len(poblaciones) < numero_mejores:
        raise Exception("El número de mejores a seleccionar nunca puede ser menor al número de poblaciones existentes")
        return
    
    # Calculamos las distancias de todas las poblaciones
    fitness_poblaciones = [fitness(individuo) for individuo in poblaciones]
    fitness_poblaciones = np.array(fitness_poblaciones)

    # Una vez tenemos las distancias de las poblaciones, seleccionamos las mejores distancias
    # con np.argpartition que separa los índices teniendo en cuenta las distancias quedándose 
    # al principio los "numero_mejores" indices y dejando detrás los mayores
    # IMPORTANTE: No los ordena
    mejores_distancias = np.argpartition(fitness_poblaciones, numero_mejores)

    # Nos quedamos con las mejores poblaciones quedándonos con las poblaciones
    # que pertenecen a los "numero_mejores" indices de poblaciones
    mejores_poblaciones = poblaciones[mejores_distancias[:numero_mejores]]

    return mejores_poblaciones

### Función del algoritmo evolutivo 
<div>Dada unas poblaciones iniciales, buscamos los mejores individuos y los mutamos o recombinamos dependiendo de unas probabilidades</div>
<div>proporcionadas por los Hiperpárametros. Si mutamos o recombinamos, no modificamos los individuos existentes, en cambio,</div>
<div>ese nuevo individuo lo añadiremos en la siguiente población.</div>
<div>Esto lo hacemos para no recombinar o mutar a partir de individuos que ya han sido recombinados o mutados en esta generación.</div>
<div>Para reducir la población de la siguiente generación, se ha creado un Hiperapárametro para decidir la manera de selección de los individuos</div>
<ul>
 <li>si es 0: seleccionamos los individuos de manera aleatoria hasta una población de tamaño N (numero_poblaciones)</li>
 <li>si es 1: seleccionados los mejores individuos hasta una población de tamaño N (numero_poblaciones)</li>
</ul>



In [61]:
def algoritmo_evolutivo(poblaciones, numero_poblaciones, probabilidad_mutacion, probabilidad_recombinacion, generaciones, numero_mejores, seleccion):
    
    for i in range(generaciones):

        mejores_poblaciones = buscar_mejores_poblaciones(poblaciones, numero_mejores)

        for poblacion in mejores_poblaciones:

            # Creamos variables para saber si ha existido recombinación o mutación
            recombinado = False
            mutado = False

            if random.random() <= probabilidad_recombinacion:
                new_poblacion = recombinacion(poblacion, random.choice(mejores_poblaciones))
                recombinado = True
                    
            if random.random() <= probabilidad_mutacion:
                new_poblacion = mutacion(poblacion)
                mutado = True
            
            # Si ha sido recombinado o mutado, las añadimos a las nuevas_poblaciones
            if recombinado or mutado:
                poblaciones = np.append(poblaciones, [np.array(new_poblacion, dtype=int)], axis=0)

        if seleccion == 0:
            np.random.shuffle(poblaciones)
            poblaciones = poblaciones[:numero_poblaciones]

        if seleccion == 1:
            poblaciones = buscar_mejores_poblaciones(poblaciones, numero_poblaciones)
        
    return min(poblaciones, key=fitness)

Hiperparámetros

In [62]:
numero_poblaciones = 10 # número de poblaciones
numero_mejores = int(numero_poblaciones/2) # número de mejores con lo que quedarse
probabilidad_recombinacion = 1 # probabilidad de recombinación 
probabilidad_mutacion = 1 # probabilidad de mutación 
generaciones = 30 # generaciones
seleccion = 1

In [63]:
poblaciones = [np.random.permutation([0, 1, 2, 3, 4]) for i in range(numero_poblaciones)]
poblaciones = np.array(poblaciones)

mejor_ruta = algoritmo_evolutivo(poblaciones, numero_poblaciones, probabilidad_mutacion, probabilidad_mutacion, generaciones, numero_mejores, seleccion)

print(f'La mejor ruta es {mejor_ruta} que serían {fitness(mejor_ruta)}km')
print("El reccorido a seguir sería", [ciudades[i] for i in mejor_ruta])

La mejor ruta es [1 0 2 3 4] que serían 181km
El reccorido a seguir sería ['Telde', 'Las Palmas de Gran Canaria', 'Arucas', 'Agaete', 'Mogán']


Código en GITHUB:
https://github.com/aaronmed/m2d/blob/master/TAREA%202%20(23-24)%20ALGORITMOS%20EVOLUTIVOS/M2D_TAREA-4-(23-24)-ALGORITMOS-EVOLUTIVOS_AaronMedinaMelian.ipynb